In [26]:
%matplotlib inline
import pymongo as mg
from matplotlib import pyplot as plt
import pandas as pd

# return a mongodb client
def get_client(usr, pwd, address, port='27017'):
    url = 'mongodb://%s:%s@%s:%s/RESTfulSwarmDB' % (usr, pwd, address, port)
    return mg.MongoClient(url)

# return a database object
def get_db(client, db_name):
    return client[db_name]

# return a collection cursor object
def get_col(db, col_name):
    return db[col_name]

# query data in a job collection
def query_col(col):
    return list(col.find({}))[0]

# get a list of job collection name
def get_all_cols(db):
    return db.collection_names()

def get_time(data):
    result = {'waiting_time': 0, 'execution_time': 0, 'total_time': 0}
    if len(data) == 0:
        return None
    submit_time = data['submit_time']
    start_time = data['start_time']
    end_time = data['end_time']
    if start_time != 0:
        end_time = data['end_time']
        if end_time == 0:
            waiting_time = start_time - submit_time
            result['waiting_time'] = waiting_time
        else:
            execution_time = end_time - start_time
            result['execution_time'] = execution_time
            total_time = end_time - submit_time
            result['total_time'] = total_time
    return result

def plot_graph(db):
    cols = get_all_cols(db)
    waiting_time = []
    execution_time = []
    total_time = []
    jobs = []
    for col in cols:
        if 'job' not in col:
            continue
        jobs.append(col)
        col_obj = get_col(db, col)
        job_info = query_col(col_obj)
        result = get_time(job_info)
        waiting_time.append(result['waiting_time'])
        execution_time.append(result['execution_time'])
        total_time.append(result['total_time'])
    
    def plot_waiting_time():
        wt = pd.Series(waiting_time, index=jobs)
        wt.plot.bar(figsize=(10, 5))
        plt.xlabel('Job')
        plt.ylabel('Waiting Time')
        plt.title('Job Waiting Time')
    
    def plot_execution_time():
        et = pd.Series(waiting_time, index=jobs)
        et.plot.bar(figsize=(10, 5))
        plt.xlabel('Job')
        plt.ylabel('Execution Time')
        plt.title('Job Execution Time')
    
    def plot_total_time():
        tt = pd.Series(total_time, index=jobs)
        tt.plot.bar(figsize=(10, 5))
        plt.xlabel('Job')
        plt.ylabel('Total Time')
        plt.title('Job Total Time')
    
    plot_waiting_time()

def main():
    client = get_client('admin', 'kzw', '129.59.107.139')
    db = get_db(client, 'RESTfulSwarmDB')
    plot_graph(db)

main()

TypeError: Empty 'DataFrame': no numeric data to plot